In [9]:
import os
import random
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.options.display.float_format = '{:.5f}'.format


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)
train=pd.read_csv("C:/Users/user/Desktop/데이콘/현대/open/결측치처리train.csv")
# 'wind_speed' 값을 기반으로 'B' 변수를 만들기 위한 함수 정의
def categorize_wind_speed(speed):
    if speed < 0.3:
        return 0
    elif 0.3 <= speed < 1.5:
        return 1
    elif 1.5 <= speed < 3.3:
        return 2
    elif 3.3 <= speed < 5.5:
        return 3
    elif 5.5 <= speed < 8:
        return 4
    elif 8 <= speed < 10.8:
        return 5
    elif 10.8 <= speed < 13.9:
        return 6
    elif 13.9 <= speed < 17.2:
        return 7
    elif 17.2 <= speed < 20.7:
        return 8
    elif 20.7 <= speed < 24.5:
        return 9
    elif 24.5 <= speed < 28.4:
        return 10
    elif 28.4 <= speed < 32.6:
        return 11
    else:
        return 12

# 'wind_speed'를 기반으로 'B' 변수 생성
train['B'] = train['wind_speed'].apply(categorize_wind_speed)

# 결과 출력
train


,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,wind_speed,B
0,TRAIN_000000,CN,EKP8,Bulk,30.73658,2020-10-15 04:03:00,Z517571,30.00000,28,73100,...,0.00166,3.04833,2020,10,15,4,3,3,3.77260,3
1,TRAIN_000001,CN,EUC8,Container,63.22042,2019-09-17 02:55:00,U467618,30.00000,15,37900,...,0.00161,17.13861,2019,9,17,2,55,1,7.42590,4
2,TRAIN_000002,CN,NGG6,Container,90.42742,2019-02-23 06:43:00,V378315,50.00000,7,115000,...,0.00174,98.82750,2019,2,23,6,43,5,0.00000,0
3,TRAIN_000003,JP,TMR7,Cargo,0.00000,2020-09-18 22:06:00,B726632,10.00000,33,1490,...,0.00007,0.00000,2020,9,18,22,6,4,8.50128,5
4,TRAIN_000004,RU,NNC2,Container,8.81373,2022-08-13 12:57:00,D215135,30.00000,10,27600,...,0.00020,96.03056,2022,8,13,12,57,5,3.00375,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367435,TRAIN_367436,CN,YRT6,Bulk,59.01818,2017-11-11 22:23:00,J661243,40.00000,13,93200,...,0.00036,65.85000,2017,11,11,22,23,5,0.41095,1
367436,TRAIN_367437,JP,QYY1,Tanker,0.00000,2022-04-29 02:58:00,D847216,10.00000,9,1280,...,0.00055,0.00000,2022,4,29,2,58,4,0.87207,1
367437,TRAIN_367438,SG,GIW5,Container,1.76863,2022-07-14 07:58:00,Q635545,30.00000,6,25000,...,0.00262,0.99750,2022,7,14,7,58,3,3.41855,3
367438,TRAIN_367439,JP,TMR7,Cargo,0.00000,2020-12-22 10:07:00,N211282,10.00000,8,2400,...,0.00007,0.00000,2020,12,22,10,7,1,4.46556,3


In [10]:
test=pd.read_csv("C:/Users/user/Desktop/데이콘/현대/open/test.csv")

In [11]:
# datetime 컬럼 처리
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

In [12]:
from sklearn.preprocessing import LabelEncoder

# 'ARI_PO' 변수 라벨 인코딩
label_encoder_ari_po = LabelEncoder()
train['ARI_PO'] = label_encoder_ari_po.fit_transform(train['ARI_PO'])
test['ARI_PO'] = label_encoder_ari_po.transform(test['ARI_PO'])
# 'ARI_CO' 변수 라벨 인코딩
label_encoder_ari_co = LabelEncoder()
train['ARI_CO'] = label_encoder_ari_co.fit_transform(train['ARI_CO'])
test['ARI_CO'] = label_encoder_ari_co.transform(test['ARI_CO'])

In [13]:
for i in test[test['U_WIND'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_PO']
    data_mean = test[(test['ARI_PO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['U_WIND'].describe()['mean']
    test.loc[i,'U_WIND'] = data_mean

In [14]:
for i in test[test['U_WIND'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_CO']
    data_mean = test[(test['ARI_CO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['U_WIND'].describe()['mean']
    test.loc[i,'U_WIND'] = data_mean

In [15]:
test['U_WIND'].isnull().sum()

752

In [16]:
for i in test[test['V_WIND'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_PO']
    data_mean = test[(test['ARI_PO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['V_WIND'].describe()['mean']
    test.loc[i,'V_WIND'] = data_mean

In [17]:
for i in test[test['V_WIND'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_CO']
    data_mean = test[(test['ARI_CO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['V_WIND'].describe()['mean']
    test.loc[i,'V_WIND'] = data_mean

In [18]:
test['V_WIND'].isnull().sum()

752

In [19]:
for i in test[test['AIR_TEMPERATURE'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_PO']
    data_mean = test[(test['ARI_PO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['AIR_TEMPERATURE'].describe()['mean']
    test.loc[i,'AIR_TEMPERATURE'] = data_mean

In [20]:
for i in test[test['AIR_TEMPERATURE'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_CO']
    data_mean = test[(test['ARI_CO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['AIR_TEMPERATURE'].describe()['mean']
    test.loc[i,'AIR_TEMPERATURE'] = data_mean

In [21]:
test['AIR_TEMPERATURE'].isnull().sum()

754

In [ ]:
for i in test[test['BN'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_PO']
    data_mean = test[(test['ARI_PO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['BN'].describe()['mean']
    test.loc[i,'BN'] = data_mean

In [ ]:
for i in test[test['BN'].isnull()].index:
    # display(test.iloc[i])
    month = test.iloc[i]['month']
    hour = test.iloc[i]['ATA_LT']
    num = test.iloc[i]['ARI_CO']
    data_mean = test[(test['ARI_CO']==num)&(test['month']==month) & (test['ATA_LT'] ==hour)]['BN'].describe()['mean']
    test.loc[i,'BN'] = data_mean

In [ ]:
test['BN'].isnull().sum()

In [ ]:
# year, month, day, hour, minute, weekday를 기반으로 시간순 정렬
test.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'weekday'], inplace=True)

# 정렬된 데이터 확인
test

In [ ]:
train['ATA'] = pd.to_datetime(train['ATA'])

conditions = [
    (train['ARI_CO'] == 'CN'),  # 'CN'인 경우 8시간 추가
    (train['ARI_CO'] == 'JP'),  # 'JP'인 경우 9시간 추가
    (train['ARI_CO'] == 'RU'),  # 'RU'인 경우 2시간 추가
    (train['ARI_CO'] == 'AU'),  # 'AU'인 경우 11시간 추가
    (train['ARI_CO'] == 'SG'),  # 'SG'인 경우 8시간 추가
    (train['ARI_CO'] == 'ZA'),  # 'ZA'인 경우 2시간 추가
    (train['ARI_CO'] == 'KR'),  # 'KR'인 경우 9시간 추가
    (train['ARI_CO'] == 'TW'),  # 'TW'인 경우 8시간 추가
    (train['ARI_CO'] == 'TT'),  # 'TT'인 경우 -4시간 추가
    (train['ARI_CO'] == 'ID'),  # 'ID'인 경우 7시간 추가
    (train['ARI_CO'] == 'BR'),  # 'BR'인 경우 -2시간 추가
    (train['ARI_CO'] == 'QA'),  # 'QA'인 경우 3시간 추가
    (train['ARI_CO'] == 'LV'),  # 'LV'인 경우 3시간 추가
    (train['ARI_CO'] == 'MZ'),  # 'MZ'인 경우 2시간 추가
    (train['ARI_CO'] == 'US'),  # 'US'인 경우 -4시간 추가
    (train['ARI_CO'] == 'IN'),  # 'IN'인 경우 5.5시간 추가
    (train['ARI_CO'] == 'UA'),  # 'UA'인 경우 3시간 추가
    (train['ARI_CO'] == 'CA'),  # 'CA'인 경우 -2.5시간 추가
    (train['ARI_CO'] == 'MY'),  # 'MY'인 경우 8시간 추가
    (train['ARI_CO'] == 'PE'),  # 'PE'인 경우 -5시간 추가
    (train['ARI_CO'] == 'VN'),  # 'VN'인 경우 7시간 추가
    (train['ARI_CO'] == 'FI'),  # 'FI'인 경우 3시간 추가
    (train['ARI_CO'] == 'CL'),  # 'CL'인 경우 -3시간 추가
    (train['ARI_CO'] == 'VE'),  # 'VE'인 경우 -4시간 추가
    (train['ARI_CO'] == 'PH')   # 'PH'인 경우 8시간 추가
]

choices = [
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=9),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=11),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=9),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=7),
    train['ATA'] + pd.Timedelta(hours=-2),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=2),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=5.5),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=-2.5),
    train['ATA'] + pd.Timedelta(hours=8),
    train['ATA'] + pd.Timedelta(hours=-5),
    train['ATA'] + pd.Timedelta(hours=7),
    train['ATA'] + pd.Timedelta(hours=3),
    train['ATA'] + pd.Timedelta(hours=-3),
    train['ATA'] + pd.Timedelta(hours=-4),
    train['ATA'] + pd.Timedelta(hours=8)
]

train['ATA'] = np.select(conditions, choices, default=train['ATA'])
train['ATA']

In [ ]:
# datetime을 여러 파생 변수로 변환
for df in [train]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

In [ ]:
test=pd.read_csv("C:/Users/user/Desktop/데이콘/현대/open/test.csv")